# MVP Análise de Dados e Boas Práticas

Nome: Daniel Waissmann Szyfman

Matrícula: 4052024001726

Dataset: [Car Evaluation Dataset](https://archive.ics.uci.edu/dataset/19/car+evaluation)

# Introdução

A avaliação de veículos é uma etapa essencial para a tomada de decisão por parte de consumidores, fabricantes e concessionárias. O *Car Evaluation Dataset*, disponibilizado pela UCI Machine Learning Repository, oferece uma base estruturada para análise automatizada de atributos de veículos, permitindo classificar o quão aceitável é um carro com base em critérios técnicos e financeiros.

Este projeto tem como objetivo aplicar técnicas de análise exploratória e pré-processamento de dados para investigar padrões dentro deste dataset e preparar os dados para futuros modelos de machine learning. Utilizaremos a linguagem Python e bibliotecas como `pandas`, `numpy` e `seaborn`, sempre buscando adotar boas práticas de organização de código e narrativa interpretativa.

Ao longo deste notebook, buscarei construir um storytelling claro: desde a definição do problema até o tratamento dos dados. A intenção é não apenas preparar os dados tecnicamente, mas também gerar insights práticos sobre os critérios que influenciam na avaliação de um veículo, e como esses critérios se relacionam entre si.

# Definição do Problema

O objetivo deste trabalho é compreender os fatores que influenciam a avaliação de veículos com base em seus atributos técnicos e econômicos. O conjunto de dados utilizado traz características como preço de compra, número de portas, capacidade de carga e segurança, entre outras variáveis categóricas.

## Tipo de Problema

Estou lidando com um problema de **classificação supervisionada**, no qual cada instância representa um veículo com características específicas, e a variável de saída indica o nível de aceitabilidade desse carro. A variável alvo possui quatro possíveis categorias: `unacc`, `acc`, `good` e `vgood`.

O desafio é identificar como os atributos fornecidos contribuem para essa avaliação final e compreender os padrões que diferenciam carros mal avaliados daqueles considerados bons ou muito bons.


## Hipóteses

Antes de iniciar a análise detalhada, levantei algumas hipóteses que pretendo verificar ao longo do trabalho:

- Carros com maior capacidade de carga e mais portas tendem a ser melhor avaliados.
- Veículos com manutenção muito cara ou baixa segurança provavelmente são classificados negativamente.
- Pode haver combinações específicas entre segurança e preço de compra que afetam diretamente a avaliação, mesmo quando os demais atributos são favoráveis.
- A segurança pode ter maior peso na classificação do que o número de passageiros ou o tamanho do porta-malas.
- Veículos baratos com boa segurança podem ter avaliações boas mesmo com menor conforto.

Essas hipóteses servirão como base para guiar a análise exploratória e a interpretação dos dados.


## Seleção de Dados

O dataset escolhido foi o *Car Evaluation Dataset*, originalmente publicado no UCI Machine Learning Repository. A base contém apenas atributos categóricos, que representam diferentes aspectos técnicos e funcionais de um veículo, e é amplamente utilizada para tarefas de classificação.

A escolha foi motivada pela simplicidade estrutural, facilidade de entendimento e riqueza de combinações possíveis entre os atributos, o que permite uma análise interpretativa eficaz e um fluxo de pré-processamento direto.


## Atributos do Dataset

A base de dados é composta pelos seguintes atributos:

- **buying**: custo de compra (valores: vhigh, high, med, low)
- **maint**: custo de manutenção (valores: vhigh, high, med, low)
- **doors**: número de portas (valores: 2, 3, 4, 5more)
- **persons**: capacidade de pessoas (valores: 2, 4, more)
- **lug_boot**: tamanho do porta-malas (valores: small, med, big)
- **safety**: nível de segurança (valores: low, med, high)

A variável alvo (que representa a classificação final do veículo) é:

- **class**: avaliação final do carro (valores: unacc, acc, good, vgood)

Todos os atributos do conjunto de dados são **categóricos com ordenação natural**. Isso significa que, apesar de representados por strings, existe uma hierarquia implícita entre os valores de cada atributo. Essa característica será essencial no momento do pré-processamento, quando precisarei aplicar uma codificação que preserve essa ordem.



# Importação das Bibliotecas Necessárias e Carga de Dados

Nesta etapa, realizarei a importação das bibliotecas essenciais para análise, visualização e preparação dos dados. Também farei o carregamento inicial do dataset para dar início à exploração.

In [7]:
# Bibliotecas para manipulação e análise de dados
import pandas as pd
import numpy as np

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações opcionais para os gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [8]:
url = "https://raw.githubusercontent.com/Szyfman/MVP_Puc_Final/main/car.csv"

In [9]:
colunas = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv(url, names=colunas)

In [10]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


# Exploração Inicial dos Dados

Com o dataset carregado, agora farei uma exploração inicial para entender melhor sua estrutura, identificar tipos de variáveis, verificar valores únicos e checar se há dados ausentes. Essas informações são importantes para planejar o pré-processamento adequado mais adiante.

In [11]:
# Dimensão do dataset
print("Dimensão do dataset:", df.shape)

# Informações gerais sobre os tipos de dados
print("\nInformações sobre colunas:")
df.info()

# Verificação de valores nulos
print("\nTotal de valores ausentes por coluna:")
print(df.isnull().sum())

Dimensão do dataset: (1728, 7)

Informações sobre colunas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB

Total de valores ausentes por coluna:
buying      0
maint       0
doors       0
persons     0
lug_boot    0
safety      0
class       0
dtype: int64


In [ ]:
# Verifico quantos valores únicos existem em cada coluna
print("Valores únicos por coluna:")
print(df.nunique())

# Frequência de valores por coluna (amostra)
for coluna in df.columns:
    print(f"\nDistribuição de valores em '{coluna}':")
    print(df[coluna].value_counts())

Valores únicos por coluna:
buying      4
maint       4
doors       4
persons     3
lug_boot    3
safety      3
class       4
dtype: int64

Distribuição de valores em 'buying':
buying
vhigh    432
high     432
med      432
low      432
Name: count, dtype: int64

Distribuição de valores em 'maint':
maint
vhigh    432
high     432
med      432
low      432
Name: count, dtype: int64

Distribuição de valores em 'doors':
doors
2        432
3        432
4        432
5more    432
Name: count, dtype: int64

Distribuição de valores em 'persons':
persons
2       576
4       576
more    576
Name: count, dtype: int64

Distribuição de valores em 'lug_boot':
lug_boot
small    576
med      576
big      576
Name: count, dtype: int64

Distribuição de valores em 'safety':
safety
low     576
med     576
high    576
Name: count, dtype: int64

Distribuição de valores em 'class':
class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64


### Considerações iniciais

O dataset possui 1.728 registros e 7 variáveis, todas categóricas. Nenhum valor ausente foi identificado, o que elimina a necessidade de tratamento de dados faltantes nesta etapa. A análise preliminar também mostrou que os atributos de entrada possuem uma distribuição de valores bastante uniforme, o que contribui para manter a diversidade nas observações.

Por outro lado, a variável alvo (`class`) apresenta um desbalanceamento considerável: mais de 70% dos veículos foram classificados como `unacc`, enquanto as classes `good` e `vgood` aparecem com frequência muito menor. Essa discrepância pode ter impacto direto em análises futuras e deve ser levada em conta ao interpretar os padrões nos dados.

Essas observações iniciais fornecem uma boa base para seguir com as estatísticas descritivas e as visualizações, aprofundando a compreensão sobre os comportamentos e relações entre as variáveis.
